In [1]:
import torch 
from torch import nn

In [2]:
mha = nn.MultiheadAttention(1,1,batch_first=True)

In [ ]:
key_padding_mask=None, need_weights=True, attn_mask=None, average_attn_weights=True, is_causal=False

In [3]:
causal_mask = torch.triu(torch.ones(5, 5), diagonal=1).bool()
pad_mask = []
for i in range(5):
    mask = [0 for _ in range(torch.randint(1,10,(1,)).item())]
    mask += [1]*(10-len(mask))
    pad_mask.append(mask)
pad_mask = torch.tensor(pad_mask,dtype=torch.bool)
pad_mask[0,1:]=1
pad_mask

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True]])

In [21]:
pad_mask = torch.tensor([[True, True, True, False, False],[False, True, True, False, False]]) 


In [28]:
x = torch.rand((2,5,1))
attn_mask = mha.merge_masks(causal_mask,pad_mask,x)[0].reshape(-1,5,5)
outputs = mha(x,x,x,attn_mask = attn_mask.masked_fill(attn_mask == 1, float('-inf')))[0]

In [30]:
pad_mask

tensor([[ True,  True,  True, False, False],
        [False,  True,  True, False, False]])

In [31]:
b, n, e, c = 2, 5, 3, 4  # batch, tokens, experts, capacity

# Example dispatch tensor: (b, n, e, c)
dispatch_tensor = torch.randint(0, 2, (b, n, e, c), dtype=torch.bool)

# Example pad mask: (b, n) - True means padding, False means real token
pad_mask = torch.randint(0, 2, (b, n), dtype=torch.bool)

# Step 1: Expand pad_mask to match dispatch_tensor shape (b, n, 1, 1)
pad_mask_expanded = pad_mask.unsqueeze(-1).unsqueeze(-1)  # Shape: (b, n, 1, 1)

# Step 2: Apply dispatch_tensor to get padding distribution
expert_pad_mask = (dispatch_tensor & pad_mask_expanded)  # Shape: (b, n, e, c)

# Step 3: Aggregate over tokens (n) and experts (e) to compute final mask
final_pad_mask = expert_pad_mask.any(dim=(1, 2))  # Shape: (b, c)

# Display results
print(final_pad_mask)

tensor([[True, True, True, True],
        [True, True, True, True]])


In [32]:
pad_mask

tensor([[ True, False, False,  True, False],
        [ True, False, False,  True,  True]])

In [37]:
with open('shakespear.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [38]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [39]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]